### Import Activities File

### Explore Activities on Strava Data

In [ ]:
import pandas as pd
df = pd.read_csv('../data/activities.csv')
df.columns

In [ ]:
# Total lines & total number of columns
df.shape

In [ ]:
# Unique Activity Types
df["Activity Type"].unique()

In [ ]:
# Pick necessary columns to work with
data = df[df.columns[[0,1,2,3,4,5,6,10,13,14,15,16,17,18,19,20,21,22,23,26,27,31,32,41,44,45,47,50]]]
data[-7:]

In [ ]:
# Filter down to only Bicycle Rides (Ride and Virtual Ride)
rideTypes = ['Virtual Ride', 'Ride']
virtual_rides = data[data['Activity Type'].isin(rideTypes)]
virtual_rides

### Extract GPX.GZ File

In [ ]:
# define input data and output data path
filepath = "../data/"
fileOutputPath= filepath + "output/"

In [ ]:
# Extract activity filename
filenamegz = data['Filename'][140]
print(filenamegz)
filename=filenamegz[:-3]
print(filename)

In [ ]:
# Extract GPX File to examine
import gzip
import shutil
with gzip.open(filepath + filenamegz, 'rb') as f_in:
    with open(fileOutputPath + filename, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
# Print a GPX data to understand 
with open(fileOutputPath + filename, 'r') as test:
    print(test.read())

### Read GPX File

In [ ]:
## Install python-benedict library
# !pip install python-benedict

In [ ]:
from benedict import benedict as bdict
# data-source can be an url, a filepath or data-string (as in this example)
data_source = fileOutputPath + filename
xmldata = bdict.from_xml(data_source)
#xmldata

In [ ]:
trk_list = xmldata['gpx.trk.trkseg.trkpt']
trk_list

In [ ]:
# Put just Location data into an array
locs=[]
for t in trk_list:
    print(f"lat: %s, lon: %s, time: %s" % (t['@lat'], t['@lon'],t['time']))
    locs.append([float(t['@lat']),float(t['@lon'])])

In [ ]:
len(locs)

### Data Visualization

In [ ]:
## Install for Data Visualization
#!pip install ipyleaflet
#!jupyter nbextension enable --py --sys-prefix ipyleaflet 
#
# or just use conda
#!conda install -c conda-forge ipyleaflet

In [ ]:
# Geo Map plot sample
from ipyleaflet import Map, Polyline

line = Polyline(
    locations=locs,
    color="green" ,
    fill=False
)

m = Map(center = ((min(locs)[0]+max(locs)[0])/2, (min(locs)[1]+max(locs)[1])/2), zoom =12)
m.add_layer(line)
m

In [ ]:
fileOutputPath

#### Power Distribution on a Ride

In [ ]:
from utils import *
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

virtualSteps = getVirtualLocations(fileOutputPath,getFilename(data['Filename'][190]))
  
dfSteps = pd.DataFrame(virtualSteps, columns =['Latitude', 'Longtitude','Elevation','Power','Date']) 
dfSteps = dfSteps.astype({"Power": int})

plt.figure(figsize=(15,8))
powerPlot = sns.lineplot(data=dfSteps, x="Date", y="Power")
powerPlot.set(xticklabels=[]) 
powerPlot.set(xlabel=None)
powerPlot.tick_params(bottom=False) 
powerPlot

In [ ]:
dfSteps['Power'].plot(kind = 'hist', figsize=(14,8))

### Investigate on Values

In [ ]:
df[['Activity Name','Moving Time', 'Max Speed','Average Speed','Weighted Average Power', 'Power Count']][170:176]

In [ ]:
#df['Weighted Average Power'][170] * df['Power Count'][170]

In [ ]:
# distance km
df['Distance.1'][170]/1000

In [ ]:
# Max km/h speed original value is m/s
# divide by 3600/1000
df['Max Speed'][170]*3.6